In [31]:
import glob
import os
import jq
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

In [ ]:
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [7]:
kb_dir = "kb"

In [13]:
all_docs = []
files = glob.glob(os.path.join(kb_dir, '*.jsonl'))

for f in files:
    loader = JSONLoader(
        file_path=f,
        jq_schema='.content',
        text_content=False,
        json_lines=True,
    )

    loaded_docs = loader.load()
    all_docs.extend(loaded_docs)
    print(f"Loaded {len(loaded_docs)} documents from {f}")

Loaded 579 documents from kb\scraped_alumni.jsonl
Loaded 900 documents from kb\scraped_calon_mahasiswa.jsonl
Loaded 2624 documents from kb\scraped_mahasiswa.jsonl
Loaded 536 documents from kb\scraped_ortu.jsonl


In [15]:
text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(all_docs)

In [ ]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large",
    model_kwargs={"device": "cuda"}
)


No sentence-transformers model found with name hkunlp/instructor-large. Creating a new one with mean pooling.


RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, maia, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: gpu

In [ ]:
db = Chroma.from_documents(
    documents=docs, 
    embedding=embeddings, 
    persist_directory='embeddings'
)

C:\Users\dheoz\AppData\Local\Temp\ipykernel_18384\3832338933.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [30]:
retriever = db.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOllama(model="llama3")
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
print("RAG Chain initialized successfully.")


RAG Chain initialized successfully.


C:\Users\dheoz\AppData\Local\Temp\ipykernel_18384\1306422039.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama3")


In [32]:
app = FastAPI()
origins = ["*"]
app.add_middleware(CORSMiddleware, allow_origins=origins, allow_credentials=True, allow_methods=["*"], allow_headers=["*"])

class ChatRequest(BaseModel):
    message: str

@app.post("/chat")
async def chat(request: ChatRequest):
    if not chain:
         return {"error": "RAG chain not initialized"}
    try:
        response = chain.invoke(request.message)
        return {"response": response}
    except Exception as e:
        print(f"Error during chain invocation: {e}")
        return {"error": "Failed to get a response from the chatbot."}

@app.get("/")
def read_root():
    return {"message": "RAG Chatbot API is running."}